# BACKTESTING STOCK MARKET SHORT PREDICTION MODEL

@author: Matheus José Oliveira dos Santos

## 1. Imports

In [1]:
import sys
import os
from datetime import date, datetime, timedelta

import pandas as pd
import numpy as np
import joblib

sys.path.append(os.getcwd()+'\\..\\src')
import db_interface
import portfolio_backtest

## 2. class inheritance

In [2]:
class Backtest(portfolio_backtest.Backtest):
    def get_data(self) -> np.array:
        print("getting data")
        _ref_date = self._date_indicator
        table_name = "financials"
        
        my_db = db_interface.DB_interface("FINANCE_DB")

        with my_db:
            for i in range(5):
                #query only brazilian stocks
                query = "SELECT * FROM {0} WHERE date = '{1}'".format(table_name, _ref_date.strftime("%Y-%m-%d"))
                df_data = my_db.read_by_command(query)
                if df_data.shape[0] == 0:
                    #if there are no data, get the day before
                    _ref_date = _ref_date - timedelta(days=1)
                else:
                    break #exit loop
                    raise ValueError("There are not enough data")
        
        #pre-processing
        df_sample = df_data.copy().iloc[:,2:-3]
        sc = joblib.load(self._sc)
        ar_sample_scaled = sc.transform(df_sample)
        
        #save BBGTicker order
        self._order_BBGTicker = df_data['BBGTicker']

        #return the sample scaled and save the order of BBGTicker
        return ar_sample_scaled

    def allocation(self, df_input:pd.DataFrame) -> pd.DataFrame:
        df_aloc = 
    
    def new_year(self) -> None:
        pass

## 3. Backtest

In [3]:
start_date = date(2022,1,1)
end_date = date(2023,8,1)
delta = timedelta(weeks=1)

list_scalers = ['sc2013.pkl','sc2014.pkl','sc2015.pkl','sc2016.pkl','sc2017.pkl','sc2018.pkl','sc2019.pkl','sc2020.pkl','sc2021.pkl','sc2022.pkl','sc2023.pkl']
list_models = ['ANN2013','ANN2014','ANN2015','ANN2016','ANN2017','ANN2018','ANN2019','ANN2020','ANN2021','ANN2022','ANN2023']

In [4]:
bt = Backtest(start_date, end_date, delta)
bt.set_date_indicator(date(2023,7,21))
bt.set_scaler(os.getcwd()+'\\..\\scalers\\'+list_scalers[0])
ar_sample_sc = bt.get_data()
#bt.executar()

getting data
connecting in: FINANCE_DB
DB Closed


C:\Users\mathe\Coding Projects\quant-finance-strategy\notebooks\..\src\db_interface.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(command, self.db)


In [5]:
df_predictions = bt.make_predictions(model_path = os.getcwd()+'\\..\\models\\'+list_models[0], ar_sample = ar_sample_sc)
print(df_predictions.head(20))

4/4 [==============================] - 0s 2ms/step
          BBGTicker  probability
0   WEGE3 BZ Equity     0.999985
1   GUAR3 BZ Equity     0.999900
2   CMIG3 BZ Equity     0.982663
3   SMTO3 BZ Equity     0.978808
4   RANI3 BZ Equity     0.276227
5   OIBR3 BZ Equity     0.031317
6   GFSA3 BZ Equity     0.013635
7   EMBR3 BZ Equity     0.010623
8   POSI3 BZ Equity     0.008574
9   PCAR3 BZ Equity     0.003797
10  CSNA3 BZ Equity     0.002977
11  SEQL3 BZ Equity     0.002407
12  CPLE3 BZ Equity     0.001515
13  PDGR3 BZ Equity     0.001471
14  AMAR3 BZ Equity     0.001465
15  TCSA3 BZ Equity     0.001073
16  MEAL3 BZ Equity     0.000995
17  SEER3 BZ Equity     0.000751
18  MTRE3 BZ Equity     0.000700
19  LJQQ3 BZ Equity     0.000623


In [6]:
df = bt.get_performance()